<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### Download Data

We will use [Walmart annual report PDFs](https://github.com/sujee/data/tree/main/data-prep-kit/walmart-reports-1) as our input data.

Feel free to substitute your data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart-10K-Reports-Optimized_2023.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart-10K-Reports-Optimized_2023.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024_copy.pdf' ))  # create a dupe file

Local file 'input/Walmart-10K-Reports-Optimized_2023.pdf' (1.61 MB) already exists. Skipping download.
Local file 'input/Walmart_2024.pdf' (4.87 MB) already exists. Skipping download.
Local file 'input/Walmart_2024_copy.pdf' (4.87 MB) already exists. Skipping download.


### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### Import Common python modules

In [4]:
import os
import sys

# Main repo root
from utils import rootdir

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

STAGE = 0

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
STAGE  += 1
# STAGE = 1  ## DEBUG

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_parquet_out")

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


01:00:45 INFO - Running locally
01:00:45 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
01:00:45 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
01:00:45 INFO - data factory data_ max_files -1, n_sample -1
01:00:45 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
01:00:45 INFO - pipeline id pipeline_id
01:00:45 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
01:00:45 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
01:00:45 INFO - actor creation delay 0
01:00:45 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
202

✅ Stage:1 completed successfully
CPU times: user 4.4 s, sys: 3.02 s, total: 7.41 s
Wall time: 15min 1s


### Inspect Generated output

Here we should see one entry per input file processed

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (3, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,Walmart_2024.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,df0d7842-b162-4511-80d9-190cb557bc34,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-11T01:08:31.867666,456.552947,Walmart_2024.pdf
1,Walmart-10K-Reports-Optimized_2023.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,82,1158,a1d5b15f-fe07-4444-a16f-f1a7b6fecff5,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-11T01:09:02.389041,487.067450,Walmart-10K-Reports-Optimized_2023.pdf
2,Walmart_2024_copy.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [8]:
STAGE  += 1
# STAGE = 2  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_chunk_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### Execute 

In [9]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

01:15:45 INFO - Running locally
01:15:45 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
01:15:45 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
01:15:45 INFO - data factory data_ max_files -1, n_sample -1
01:15:45 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
01:15:45 INFO - pipeline id pipeline_id
01:15:45 INFO - code location None
01:15:45 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
01:15:45 INFO - actor creation delay 0
01:15:45 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'r

✅ Stage:2 completed successfully
CPU times: user 893 ms, sys: 229 ms, total: 1.12 s
Wall time: 18.3 s


### Inspect Generated output

We would see documents are split into many chunks

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(max(3, output_df.shape[0]))

Files processed : 3
Chunks created : 1,973
Input data dimensions (rows x columns)=  (3, 12)
Output data dimensions (rows x columns)=  (1973, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
1599,Walmart_2024_copy.pdf,100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf,"( ' , ( % # - , . % - , ( ) + - # ( ' ,\n( 9 H...",$.tables[17],45,"[35.58596802, 645.70812988, 549.18774414, 724...."
1735,Walmart_2024_copy.pdf,100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf,"# + % . , . + & ' -\n. < 9 C A D 5 B M F 9 7 C...",$.main-text[642],63,"[35.25471497, 124.5, 550.93585205, 185.64001465]"
1796,Walmart_2024_copy.pdf,100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf,% @ E 6 * 9 @ C E E 6 C > @ C C @ H : ? 8 D 2 ...,$.main-text[742],71,"[35.35443878, 679.0, 549.61694336, 704.14001465]"
1384,Walmart_2024_copy.pdf,100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf,F > 2 ? 2 A : E 2 = $ 2 ? 2 8 6 > 6 ? E\n. 9 7...,$.main-text[182],15,"[35.34682083, 335.99993896, 540.63574219, 371...."
545,Walmart_2024.pdf,100,83,1058,df0d7842-b162-4511-80d9-190cb557bc34,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-11T01:08:31.867666,456.552947,Walmart_2024.pdf,"# 2 ( * 6 8 & . 2 "" & < 3 7 . 8 . 3 2 7\n. < 9...",$.main-text[820],76,"[35.27887726, 296.0, 549.15588379, 321.14001465]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,Walmart_2024_copy.pdf,100,83,1058,86f5aff5-ffd1-4a24-b67d-5f340529b925,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-11T01:15:33.376308,421.475128,Walmart_2024_copy.pdf,"+ $ ) + & ) * / , + - & ) * % & ) ' & ) + & - ...",$.main-text[914],84,"[35.42567444, 631.0, 548.65386963, 668.14001465]"
500,Walmart_2024.pdf,100,83,1058,df0d7842-b162-4511-80d9-190cb557bc34,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-11T01:08:31.867666,456.552947,Walmart_2024.pdf,% @ E 6 * 9 @ C E E 6 C > @ C C @ H : ? 8 D 2 ...,$.main-text[745],71,"[35.17734146, 515.39996338, 542.61682129, 588...."
1136,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,a1d5b15f-fe07-4444-a16f-f1a7b6fecff5,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-11T01:09:02.389041,487.067450,Walmart-10K-Reports-Optimized_2023.pdf,"As of January 31,\nThe aggregate annual lease ...",$.main-text[802],73,"[46.61784363, 518.88317871, 334.92474365, 530...."
353,Walmart_2024.pdf,100,83,1058,df0d7842-b162-4511-80d9-190cb557bc34,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-11T01:08:31.867666,456.552947,Walmart_2024.pdf,* F > > 2 C J @ 7 C : E : 4 2 = 4 4 @ F ? E : ...,$.main-text[520],50,"[35.10198975, 586.0, 544.31689453, 623.14001465]"


## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [11]:
STAGE  += 1
# STAGE  = 3  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_ededupe_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_ededupe_out'


### Execute 

In [12]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

ImportError: cannot import name 'SnapshotUtils' from 'data_processing.data_access' (/home/sujee/apps/anaconda3/envs/data-prep-kit-6-src-dev/lib/python3.11/site-packages/data_processing/data_access/__init__.py)

### Inspect Generated output

In [13]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(max(3, output_df.shape[0]))

ValueError: No objects to concatenate

## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [ ]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  += 1
# STAGE  = 4  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_id_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

In [ ]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(max(3, output_df.shape[0]))

## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [ ]:
## Input to this component is the output of doc_id generator component. 

STAGE  += 1
# STAGE  = 5  ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_fdedupe_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(max(3, output_df.shape[0]))

## Step-7: Document Quality

### Set Input/output Folder

In [ ]:
STAGE  += 1
# STAGE  = 6 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_doc_quality_out")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

### Execute 

In [ ]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_ray import DocQualityRayTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocQualityRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(max(3, output_df.shape[0]))

## Step-8:   Text encoding

Encode text for the vector storage.

In [ ]:
STAGE  += 1
# STAGE  = 7 ## DEBUG

input_folder = output_folder # previous output folder is the input folder for the current stage
output_folder =  os.path.join(MY_CONFIG.OUTPUT_FOLDER, f"{STAGE:02}_encoder_out")

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

In [ ]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

### Inspect Generated output

In [ ]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(max(3, output_df.shape[0]))

## Step-9: Copy output to final output dir

In [ ]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")